In [1]:
%run '~/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate.py'

In [2]:
from sklearn.multioutput import MultiOutputRegressor

est = xgb.XGBRFRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=100,
    max_depth=5,
    device='cuda',
    tree_method='hist'
)

model = MultiOutputRegressor(est)
model.fit(x_train, y_train, eval_set=[(x_val, y_val)])
predictions = model.predict(x_test)


XGBoostError: [19:14:19] /home/conda/feedstock_root/build_artifacts/xgboost-split_1764148514279/work/src/common/error_msg.cc:109: Invalid `base_score`, it should match the number of outputs for multi-class/target models. `base_score` len: 1, `n_targets`: 16101
Stack trace:
  [bt] (0) /home/christianl/miniconda3/envs/remote_training/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x6e) [0x7419fa0d796e]
  [bt] (1) /home/christianl/miniconda3/envs/remote_training/lib/libxgboost.so(xgboost::error::InvalidIntercept(int, unsigned int, unsigned long)+0x2d7) [0x7419fa1d1a87]
  [bt] (2) /home/christianl/miniconda3/envs/remote_training/lib/libxgboost.so(xgboost::LearnerModelParamLegacy::Validate(xgboost::Context const*) const+0x22) [0x7419fa521cb2]
  [bt] (3) /home/christianl/miniconda3/envs/remote_training/lib/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, std::shared_ptr<xgboost::DMatrix>)+0x72a) [0x7419fa52c35a]
  [bt] (4) /home/christianl/miniconda3/envs/remote_training/l